<a href="https://colab.research.google.com/github/saks0106/DL_Frequent_Lookout/blob/main/CNN%2BRNN%2BLSTM%2BGRU_with_Text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# https://www.kaggle.com/shivamkushwaha/bbc-full-text-document-classification
!wget -nc https://lazyprogrammer.me/course_files/nlp/bbc_text_cls.csv

--2023-10-11 11:32:43--  https://lazyprogrammer.me/course_files/nlp/bbc_text_cls.csv
Resolving lazyprogrammer.me (lazyprogrammer.me)... 172.67.213.166, 104.21.23.210, 2606:4700:3031::6815:17d2, ...
Connecting to lazyprogrammer.me (lazyprogrammer.me)|172.67.213.166|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5085081 (4.8M) [text/csv]
Saving to: ‘bbc_text_cls.csv’

bbc_text_cls.csv    100%[===================>]   4.85M  5.16MB/s    in 0.9s    

2023-10-11 11:32:45 (5.16 MB/s) - ‘bbc_text_cls.csv’ saved [5085081/5085081]



In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Input, GlobalMaxPooling1D
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Embedding
from tensorflow.keras.models import Model
from tensorflow.keras.losses import SparseCategoricalCrossentropy

In [ ]:
df = pd.read_csv('bbc_text_cls.csv')
df.head()

,text,labels
0,Ad sales boost Time Warner profit\n\nQuarterly...,business
1,Dollar gains on Greenspan speech\n\nThe dollar...,business
2,Yukos unit buyer faces loan claim\n\nThe owner...,business
3,High fuel prices hit BA's profits\n\nBritish A...,business
4,Pernod takeover talk lifts Domecq\n\nShares in...,business


In [ ]:
df['targets'] = df['labels'].astype("category").cat.codes
# number of classes
K = df['targets'].max() + 1
K

5

In [ ]:
df_train, df_test = train_test_split(df, test_size=0.3)
# Convert sentences to sequences
MAX_VOCAB_SIZE = 2000
tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE)
tokenizer.fit_on_texts(df_train['text'])
sequences_train = tokenizer.texts_to_sequences(df_train['text'])
sequences_test = tokenizer.texts_to_sequences(df_test['text'])

In [ ]:
# get word -> integer mapping
word2idx = tokenizer.word_index
V = len(word2idx)
print('Found %s unique tokens.' % V)

Found 27616 unique tokens.


In [ ]:
# pad sequences so that we get a N x T matrix
data_train = pad_sequences(sequences_train)
print('Shape of data train tensor:', data_train.shape)

# get sequence length
T = data_train.shape[1]

Shape of data train tensor: (1557, 2839)


In [ ]:
data_test = pad_sequences(sequences_test, maxlen=T)
print('Shape of data test tensor:', data_test.shape)

Shape of data test tensor: (668, 2839)


In [ ]:
# Create the model

# We get to choose embedding dimensionality
D = 50

# Note: we actually want to the size of the embedding to (V + 1) x D,
# because the first index starts from 1 and not 0.
# Thus, if the final index of the embedding matrix is V,
# then it actually must have size V + 1.

i = Input(shape=(T,))
x = Embedding(V + 1, D)(i)
x = Conv1D(32, 3, activation='relu')(x)
# x = MaxPooling1D(3)(x)
# x = Conv1D(64, 3, activation='relu')(x)
# x = MaxPooling1D(3)(x)
# x = Conv1D(128, 3, activation='relu')(x)
x = GlobalMaxPooling1D()(x)
x = Dense(K)(x)

model = Model(i, x)

In [ ]:
# Create the model for SimpleRNN, LSTM,GRU

# We get to choose embedding dimensionality
#D = 20

# Note: we actually want to the size of the embedding to (V + 1) x D,
# because the first index starts from 1 and not 0.
# Thus, if the final index of the embedding matrix is V,
# then it actually must have size V + 1.

# i = Input(shape=(T,))
# x = Embedding(V + 1, D)(i)
# x = LSTM(32, return_sequences=True)(x)
# x = GlobalMaxPooling1D()(x)
# x = Dense(K)(x)

# model = Model(i, x)

In [ ]:
# Compile and fit
model.compile(
  loss=SparseCategoricalCrossentropy(from_logits=True),
  optimizer='adam',
  metrics=['accuracy']
)


print('Training model...')
r = model.fit(
  data_train,
  df_train['targets'],
  epochs=50,
  validation_data=(data_test, df_test['targets'])
)

Training model...
Epoch 1/50
49/49 [==============================] - 9s 155ms/step - loss: 1.5631 - accuracy: 0.4232 - val_loss: 1.4749 - val_accuracy: 0.6228
Epoch 2/50
49/49 [==============================] - 12s 253ms/step - loss: 1.3223 - accuracy: 0.6943 - val_loss: 1.1531 - val_accuracy: 0.7156
Epoch 3/50
49/49 [==============================] - 10s 213ms/step - loss: 0.9191 - accuracy: 0.8028 - val_loss: 0.7425 - val_accuracy: 0.8293
Epoch 4/50
49/49 [==============================] - 7s 149ms/step - loss: 0.5306 - accuracy: 0.8940 - val_loss: 0.4547 - val_accuracy: 0.8997
Epoch 5/50
49/49 [==============================] - 7s 151ms/step - loss: 0.3093 - accuracy: 0.9409 - val_loss: 0.3182 - val_accuracy: 0.9192
Epoch 6/50
49/49 [==============================] - 8s 163ms/step - loss: 0.2020 - accuracy: 0.9595 - val_loss: 0.2506 - val_accuracy: 0.9266
Epoch 7/50
49/49 [==============================] - 7s 136ms/step - loss: 0.1430 - accuracy: 0.9685 - val_loss: 0.2167 - val_acc

In [ ]:
# Plot loss per iteration
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend();

In [ ]:
# Plot accuracy per iteration
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend();